In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.datasets import make_circles, make_moons
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Optioneel: GIF support
try:
    import imageio.v2 as imageio
    HAS_IMAGEIO = True
except Exception:
    HAS_IMAGEIO = False


In [2]:
# Parameters (pas aan wat je wilt)
dataset   = "circles"   # of "moons"
samples   = 800
noise     = 0.1
seed      = 42
frames    = [1, 10, 50, 200]   # epochs waarop je een plot wilt
hidden    = [16, 16]
outdir    = Path("nn_boundary_viz_simple")
outdir.mkdir(parents=True, exist_ok=True)


In [3]:
# Data rechtstreeks met scikit-learn
if dataset == "circles":
    X, y = make_circles(n_samples=samples, noise=noise, factor=0.5, random_state=seed)
elif dataset == "moons":
    X, y = make_moons(n_samples=samples, noise=noise, random_state=seed)
else:
    raise ValueError("dataset moet 'circles' of 'moons' zijn")

X.shape, y.shape


((800, 2), (800,))

In [7]:
# Model: scaler + MLP; 1 epoch per fit() zodat we frames kunnen pakken
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=tuple(hidden),
        activation="tanh",
        solver="adam",
        alpha=1e-4,
        batch_size=128,
        learning_rate_init=0.01,
        max_iter=1,          # 1 extra iter per fit-call
        warm_start=True,     # training gaat door
        random_state=seed,
        verbose=True,
    )),
])
clf


Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(activation='tanh', batch_size=128,
                               hidden_layer_sizes=(16, 16),
                               learning_rate_init=0.01, max_iter=1,
                               random_state=42, verbose=True,
                               warm_start=True))])

In [5]:
# Eén compacte plotfunctie (één plot, geen expliciete kleuren)
def plot_decision_boundary(clf, X, y, title, path):
    x_min, x_max = X[:, 0].min() - 1.0, X[:, 0].max() + 1.0
    y_min, y_max = X[:, 1].min() - 1.0, X[:, 1].max() + 1.0
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 400),
        np.linspace(y_min, y_max, 400)
    )
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1].reshape(xx.shape)

    plt.figure(figsize=(6, 5))
    plt.contourf(xx, yy, Z, alpha=0.6, levels=30)      # geen kleuren forceren
    plt.scatter(X[y == 0, 0], X[y == 0, 1], s=15, label="Class 0")
    plt.scatter(X[y == 1, 0], X[y == 1, 1], s=15, label="Class 1")
    plt.title(title)
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches="tight")
    plt.close()


In [ ]:
# Trainen + frames wegschrijven
frames = sorted(set(frames))
max_epoch = frames[-1]
saved = []

for epoch in range(1, max_epoch + 1):
    clf.fit(X, y)
    if epoch in frames:
        p = outdir / f"decision_boundary_epoch_{epoch:04d}.png"
        plot_decision_boundary(clf, X, y, f"{dataset.capitalize()} — Epoch {epoch}", p)
        saved.append(p)

print("Saved frames:")
for p in saved:
    print(p)


In [ ]:
# (Optioneel) GIF bouwen en laatste frame inline tonen (in notebooks)
from IPython.display import Image, display

gif_path = None
if HAS_IMAGEIO and len(saved) > 1:
    images = [imageio.imread(p) for p in saved]
    gif_path = outdir / "training_progress.gif"
    imageio.mimsave(gif_path, images, duration=0.8)
    print("Animated GIF:", gif_path)

if saved:
    display(Image(filename=str(saved[-1])))
